# Making Unstructured Mesh

Our preferred method to make unstructured mesh is with [gmsh](https://gitlab.onelab.info/gmsh/gmsh).  There are many [tutorials](http://gmsh.info/doc/texinfo/gmsh.html) and [examples](https://gitlab.onelab.info/gmsh/gmsh/-/tree/master/examples) available.  Mesh built with gmsh can be created in many ways including using the gmsh interactive interface, by creating geo files and converting them to mesh files with gmsh, by importing the gmsh module into python, or by importing pygmsh into python.  

Geo files can be easily processed to add topography or for merging with other mesh.

Create Mesh<br>
[Method 1 with *esys-finley* Rectangle](#Rectangle)<br>
[Method 2 import pygmsh into python](#pygmsh)<br>
[Method 3 import gmsh into python - allows easy element length variability](#importGmsh)<br>
[Method 4 geo file, with tagging](#geofileTagging)<br>
[Complicated 3D geo file, Core Ground, Buffer Ground and Air regions](#airGround)<br>

Load Mesh<br>
[Loading mesh into *esys-escript*](#LoadMesh)<br> 
[make a fly file](#mkFly)<br>
[Making the fly, loading the fly with dirac points, using the tags for masks](#diracMasks)<br>


<a id = 'Rectangle'></a>  
### Method 1 with *esys-finley* Rectangle 
 - uniform mesh size
 - mesh made within esys-escript


In [46]:
from esys.escript import *
from esys.finley import Rectangle
domain = Rectangle(l0=1.,l1=1.,n0=11, n1=11)

<a id = 'pygmsh'></a> 
### Method 2 import pygmsh into python 
  - uniform mesh size
  - saves a .msh file
  - refinement of the mesh is possible before the mesh is made.  See 

In [47]:
import pygmsh

with pygmsh.occ.Geometry() as geom:
    geom.characteristic_length_min = 0.1
    geom.characteristic_length_max = 0.1

    rectangle = geom.add_rectangle([0.0, 0.0, 0.0], 1.0, 1.0)
    mesh = geom.generate_mesh(dim=2)

    gmsh.write("sqModel.msh")

<figure>
    <center>
    <img src="figures/UnitSquare.png", width="400">
    </center>
    <center>
    <caption>Unit Square with element size 0.1.
        </caption></center>
 </figure>
 Alternatively, can be created using the polygon function.

In [48]:
import pygmsh
with pygmsh.geo.Geometry() as geom:
    geom.add_polygon(
        [[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0],],
        mesh_size=0.1,
    )
    mesh = geom.generate_mesh()
    gmsh.write("unit_square.msh")

Can add refinement
<figure>
    <center>
    <img src="figures/UnitSquareRefined.png", width="400">
    </center>
    <center>
    <caption>Unit Square with element size 0.1, refined at top edge, 0.01 on the left nd 0.02 on the right.
        </caption></center>
 </figure>

In [49]:
import pygmsh

with pygmsh.geo.Geometry() as geom:
    Usqr = geom.add_polygon(
        [[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0],],
        mesh_size=0.1,
    )
    field0 = geom.add_boundary_layer(
        edges_list=[Usqr.curves[0]],
        lcmin=0.01,
        lcmax=0.1,
        distmin=0.0,
        distmax=0.3,
    )
    field1 = geom.add_boundary_layer(
        nodes_list=[Usqr.points[2]],
        lcmin=0.005,
        lcmax=0.1,
        distmin=0.4,
        distmax=0.5,
    )
    geom.set_background_mesh([field0, field1], operator="Min")
 
    mesh = geom.generate_mesh()
    gmsh.write("refSquare.msh")

### Can combine regions in pygmsh (or is gmsh or with a geo file). 
<figure>
    <center>
    <img src="figures/SquareRectangle.png", width="400">
    </center>
    <center>
    <caption>Unit Square with element size 0.1, a refined edge and a refined point, and attached Rectangle.
        </caption></center>
 </figure>

In [50]:
import pygmsh

with pygmsh.geo.Geometry() as geom:
    Usqr = geom.add_polygon(
        [
            [0.0, 0.0],
            [1.0, 0.0],
            [1.0, 1.0],
            [0.0, 1.0],
        ],
        mesh_size=0.1,
    )
    field0 = geom.add_boundary_layer(
        edges_list=[Usqr.curves[0]],
        lcmin=0.01,
        lcmax=0.1,
        distmin=0.0,
        distmax=0.3,
    )
    field1 = geom.add_boundary_layer(
        nodes_list=[Usqr.points[2]],
        lcmin=0.005,
        lcmax=0.1,
        distmin=0.4,
        distmax=0.5,
    )
    geom.set_background_mesh([field0, field1], operator="Min")
    Usqr2 = geom.add_polygon(
        [
            [1.0, 0.0],
            [1.5, 0.0],
            [1.5, 1.0],
            [1.0, 1.0],
        ],
        mesh_size=0.1,
    )
    geom.set_background_mesh([field0, field1], operator="Min") 
    mesh = geom.generate_mesh()
    gmsh.write("twoRegions.msh")

<a id = 'importGmsh'></a>
### Method 3 import gmsh into python - allows easy element length variability
 - refinement by specifying element length at corner nodes
 - output ia a .msh file  
<figure>
    <center>
    <img src="figures/Square.png", width="400">
    </center>
    <center>
    <caption>Unit Square with element size 0.1, refined at top edge, 0.01 on the left and 0.02 on the right.
        </caption></center>
</figure>

In [51]:
# Import modules:
import gmsh
import sys
 
# Initialize gmsh:
gmsh.initialize()

# Points
lc=0.1;
point1 = gmsh.model.geo.add_point(0, 0, 0, lc)
point2 = gmsh.model.geo.add_point(1, 0, 0, lc)
point3 = gmsh.model.geo.add_point(1, 1, 0, 0.2*lc)
point4 = gmsh.model.geo.add_point(0, 1, 0, 0.1*lc)

# Lines
line1 = gmsh.model.geo.add_line(point1, point2)
line2 = gmsh.model.geo.add_line(point2, point3)
line3 = gmsh.model.geo.add_line(point3, point4)
line4 = gmsh.model.geo.add_line(point4, point1)

# Surface
face = gmsh.model.geo.add_curve_loop([line1, line2, line3, line4])
gmsh.model.geo.add_plane_surface([face])

# Make model, mesh, and write msh file
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate()
gmsh.write("pythonGmshSquare.msh")

<a id = 'geofileTagging'></a>
### Method 4 geo file, with tagging
 - refinement by element length at corner nodes
 - tagging of regions, surfaces or lines
 - output is a .geo file
 - geo file can be modified
 - use gmsh to convert to a .msh file

<figure>
    <center>
    <img src="figures/Square.png", width="400">
    </center>
    <center>
    <caption>Unit Square with element size 0.1, refined at top edge, 0.01 on the left nd 0.02 on the right.
        </caption></center>
 </figure>
 
 To make a 2D mesh, create a geofile with
  1. points  $\qquad~$   {x, y, z, elementlength}
  2. lines     $\quad\quad~~~$ {first point, second point}.  Direction is important.
  3. line loop $\quad~~$ {line1, line2, ...lineN}  must be a loop,  - to reverse line direction 
  4. surface  $\quad~~~$  {line loop }
  5. tagging  $\quad~~~$ optional labelling of different parts or points in the domain
  
The file can be opend by the gmsh application or in the terminal with <br>
**gmsh -2 -o square.msh square.geo** <br>
where -2 menas a 2 dimensional mesh, -o square.msh is the output name and square.geo is the input geo file name.

// Points<br>
elSize = 0.1;<br>
Point(1) = {0, 0, 0, elSize };<br>
Point(2) = {1, 0, 0, elSize };<br>
Point(3) = {1, 1, 0, 0.2$*$elSize };<br>
Point(4) = {0, 1, 0, 0.1$*$elSize };

// Lines<br>
Line(1) = {1, 2};<br>
Line(2) = {2, 3};<br>
Line(3) = {3, 4};<br>
Line(4) = {4, 1};

// Surface<br>
Line Loop(1) = {1:4};   
Surface(1) = {1};

// Tagging<br>
Physical Line("Top") = {3};<br>
Physical Line("Base") = {1};<br>
Physical Surface("All") = {1};

## pygmsh cube 

In [52]:
import pygmsh

mesh_size = 0.1
geom = pygmsh.occ.Geometry()
model3D = geom.__enter__()
box0 =  model3D.add_box([0.0, 0, 0], [1, 1, 1])
geom.generate_mesh(dim=3)
gmsh.write("cubepygmsh.msh")
model3D.__exit__()

# Unit Cube using a geo file and gmsh with local refinement specified by node element length
<figure>
    <center>
    <img src="figures/gmshCube.png", width="400"><img src="figures/gmshCubeSurfaceMesh.png", width="400">
    </center>
    <center>
    <caption>Unit cube.
        </caption></center>
 </figure>
 
  To make a mesh, create a geofile with
  1. points  $\qquad~$   {x, y, z, elementlength}
  2. lines     $\quad\quad~~~$ {first point, second point}.  Direction is important.
  3. line loop $\quad~~$ {line1, line2, ...lineN}  must be a loop,  
  $\qquad\qquad~$ - to reverse line direction, outward normal using the right hand screw rule
  4. surface  $\qquad$  from the line loop 
  5. surface loop
  6. volume  $\qquad~$  from the surface loop
  7. tagging  $\qquad$ optional labelling of different parts or points in the domain
  
The file can be opend by the gmsh application or in the terminal with **gmsh -3 -o cube.msh cube.geo**


// Points
<br>
lc=0.1;

// Base
<br>
Point(1) = {0, 0, 0, 0.2$*$lc};
<br>
Point(2) = {1, 0, 0, lc};
<br>
Point(3) = {1, 1, 0, lc};
<br>
Point(4) = {0, 1, 0, lc};
<br>
// Top
<br>
Point(5) = {0, 0, 1, 0.2$*$lc};
<br>
Point(6) = {1, 0, 1, lc};
<br>
Point(7) = {1, 1, 1, lc};
<br>
Point(8) = {0, 1, 1, lc};

// Lines
<br>
// Base
<br>
Line(1) = {1, 2};<br>
Line(2) = {2, 3};<br>
Line(3) = {3, 4};<br>
Line(4) = {4, 1};<br>
// Top<br>
Line(5) = {5, 6};<br>
Line(6) = {6, 7};<br>
Line(7) = {7, 8};<br>
Line(8) = {8, 5};<br>
// Sides<br>
Line(9) = {1, 5};<br>
Line(10) = {2, 6};<br>
Line(11) = {3, 7};<br>
Line(12) = {4, 8};<br>

// Surfaces<br>
// Base<br>
Line Loop(1) = {-1,-2,-3,-4};<br>
Surface(1) = {1};<br>
// Top<br>
Line Loop(2) = {5:8};<br>
Surface(2) = {2};<br>
// Front<br>
Line Loop(3) = {1,10,-5,-9};<br>
Surface(3) = {3};<br>
// Back<br>
Line Loop(4) = {3, 12, -7, -11};<br>
Surface(4) = {4};<br>
// Left<br>
Line Loop(5) = {4, 9, -8, -12};<br>
Surface(5) = {5};<br>
// Right<br>
Line Loop(6) = {2, 11,-6, -10};<br>
Surface(6) = {6};<br>

// Volume<br>
Surface Loop(1) = {1:6};<br>
Volume(1) = {1}; <br>

// Tagging<br>
Physical Surface("Top")={2};<br>
Physical Surface("Base")={1};<br>
Physical Volume("All")={1};

<a id = 'airGround'></a>
# Complicated 3D geo file, Core Ground, Buffer Ground and Air regions
Often in geophysical inversions, more complicated mesh are needed.  This is a mesh that has a ground region split into a core region and a buffer region and an air layer. Also added to the mesh are 10 sensor locations.  The different regions are tagged in the mesh.  
<figure>
    <center>
    <img src="figures/interface.png", width="400"><img src="figures/air_ground.png", width="400">
    </center>
    <center>
    <caption>Air - Ground Interface and surface mesh.
        </caption></center>
 </figure>

## air_ground.geo

// Domain Constants<br>
// Core<br>
Cxmin = -1000.;    // min x<br>
Cxmax =  1000.;    // max x<br>
Cymin = -1500.;    // min y<br>
Cymax =  1500.;    // max y<br>
Czmin = -1500.;    // min z<br>
GrndL =  0.;       // Ground Level<br>

// Buffer<br>
Bxmin = Cxmin - 1000.;   // min x<br>
Bxmax = Cxmax + 1000.;   // max x<br>
Bymin = Cymin - 1000.;   // min y<br>
Bymax = Cymax + 1000.;   // max y<br>
Bzmin = Czmin - 1000.;   // min z<br>

// Air Layer<br>
airHt = 1000.;<br>

// Mesh element sizes<br>
//Core<br>
MCground = 100.;     // at ground level<br>
MCbase = 200.;       // at base of core<br>

// Buffer<br>
MBbase = 600.;<br>
MBground = 300.;<br>

// Air<br>
MAir = 600.;<br>

// sensor mesh length<br>
MSens = 10;<br>

//------------------------------------------------------<br>
// Points<br>

// CORE<br>
// Base<br>
Point(1)  = {Cxmin, Cymin, Czmin, MCbase};<br>
Point(2)  = {Cxmax, Cymin, Czmin, MCbase};<br>
Point(3)  = {Cxmax, Cymax, Czmin, MCbase};<br>
Point(4)  = {Cxmin, Cymax, Czmin, MCbase};<br>
// Ground<br>
Point(5)  = {Cxmin, Cymin, 0., MCground};<br>
Point(6)  = {Cxmax, Cymin, 0., MCground};<br>
Point(7)  = {Cxmax, Cymax, 0., MCground};<br>
Point(8)  = {Cxmin, Cymax, 0., MCground};<br>

// BUFFER<br>
// Base<br>
Point(9)  = {Bxmin, Bymin, Bzmin, MBbase};<br>
Point(10)  = {Bxmax, Bymin, Bzmin, MBbase};<br>
Point(11)  = {Bxmax, Bymax, Bzmin, MBbase};<br>
Point(12)  = {Bxmin, Bymax, Bzmin, MBbase};<br>
// Ground<br>
Point(13)  = {Bxmin, Bymin, 0., MBground};<br>
Point(14)  = {Bxmax, Bymin, 0., MBground};<br>
Point(15)  = {Bxmax, Bymax, 0., MBground};<br>
Point(16)  = {Bxmin, Bymax, 0., MBground};<br>

// AIR Layer<br>
Point(17)  = {Bxmin, Bymin, airHt, MAir};<br>
Point(18)  = {Bxmax, Bymin, airHt, MAir};<br>
Point(19)  = {Bxmax, Bymax, airHt, MAir};<br>
Point(20)  = {Bxmin, Bymax, airHt, MAir};<br>

//------------------------------------------------------<br>
// Lines<br>

// CORE<br>
// Base<br>
Line(1) = {1, 2};<br>
Line(2) = {2, 3};<br>
Line(3) = {3, 4};<br>
Line(4) = {4, 1};<br>
// Top<br>
Line(5) = {5, 6};<br>
Line(6) = {6, 7};<br>
Line(7) = {7, 8};<br>
Line(8) = {8, 5};<br>
// Sides<br>
Line(9) = {1, 5};<br>
Line(10) = {2, 6};<br>
Line(11) = {3, 7};<br>
Line(12) = {4, 8};<br>

// BUFFER<br>
Line(13) = {9, 10};<br>
Line(14) = {10, 11};<br>
Line(15) = {11, 12};<br>
Line(16) = {12, 9};<br>
// Top<br>
Line(17) = {13, 14};<br>
Line(18) = {14, 15};<br>
Line(19) = {15, 16};<br>
Line(20) = {16, 13};<br>
// Sides<br>
Line(21) = {9, 13};<br>
Line(22) = {10, 14};<br>
Line(23) = {11, 15};<br>
Line(24) = {12, 16};<br>

// AIR<br>
// Top<br>
Line(25) = {17, 18};<br>
Line(26) = {18, 19};<br>
Line(27) = {19, 20};<br>
Line(28) = {20, 17};<br>
// Sides<br>
Line(29) = {13, 17};<br>
Line(30) = {14, 18};<br>
Line(31) = {15, 19};<br>
Line(32) = {16, 20};<br>

//------------------------------------------------------<br>
// Surfaces<br>
// all horizontal surfaces face down for z < 0 and up for z>=0<br>
// CORE<br>
// core horizontal surfaces<br>
// Surfaces<br>
// Base<br>
Line Loop(1) = {-1,-2,-3,-4};<br>
Surface(1) = {1};<br>
Physical Surface("coreBase") = {1};<br>
// Top<br>
Line Loop(2) = {5:8};<br>
Surface(2) = {2};<br>
Physical Surface("coreInterface") = {2};<br>
// Front<br>
Line Loop(3) = {1,10,-5,-9};<br>
Surface(3) = {3};<br>
// Back<br>
Line Loop(4) = {3, 12, -7, -11};<br>
Surface(4) = {4};<br>
// Left<br>
Line Loop(5) = {4, 9, -8, -12};<br>
Surface(5) = {5};<br>
// Right<br>
Line Loop(6) = {2, 11,-6, -10};<br>
Surface(6) = {6};<br>

// BUFFER<br>
// Base<br>
Line Loop(7) = {-13,-14,-15,-16};<br>
Surface(7) = {7};<br>
Physical Surface("BufferBase") = {7};<br>
// Top<br>
Line Loop(8) = {17:20};<br>
Surface(8) = {8,-2};<br>
Physical Surface("BufferInterface") = {8};<br>
// Front<br>
Line Loop(9) = {13, 22,-17,-21};<br>
Surface(9) = {9};<br>
// Back<br>
Line Loop(10) = {15, 24, -19, -23};<br>
Surface(10) = {10};<br>
// Left<br>
Line Loop(11) = {16, 21, -20, -24};<br>
Surface(11) = {11};<br>
// Right<br>
Line Loop(12) = {14, 23, -18, -22};<br>
Surface(12) = {12};<br>

// AIR<br>
// Top<br>
Line Loop(13) = {25:28};<br>
Surface(13) = {13};<br>
Physical Surface("Top") = {13};<br>
// Front<br>
Line Loop(14) = {17, 30, -25, -29};<br>
Surface(14) = {14};<br>
// Back<br>
Line Loop(15) = {19, 32, -27, -31};<br>
Surface(15) = {15};<br>
// Left<br>
Line Loop(16) = {20, 29, -28, -32};<br>
Surface(16) = {16};<br>
// Right<br>
Line Loop(17) = {18, 31, -26, -30};<br>
Surface(17) = {17};<br>

//  SENSOR LOCATIONS<br>
k=newp;<br>
Point(k)={10.,-100., 0., MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={100., 100., 0., MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={150., -500., 0., MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={500.,-100., 0., MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={-500., 120, 0., MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={150.,-600., 0., MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={300., -700., 0., MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={100., 800., 0.,MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={150., 650., 0., MSens};<br>
Point{k} In Surface{2};<br>
k=newp;<br>
Point(k)={-300., 1000., 0., MSens};<br>
Point{k} In Surface{2};<br>

// VOLUMES<br>
// core <br>
Surface Loop(18) = {1:6};<br>
Volume(1) = {18}; <br>
Physical Volume("coreGround") = {1};<br>
// buffer <br>
Surface Loop(19) = {7:12,-1,-3,-4,-5,-6};<br>
Volume(2) = {19};<br>
Physical Volume("buffer") = {2};<br>
// air<br>
Surface Loop(20) = {2,8,13:17};<br>
Volume(3) = {20}; <br>
Physical Volume("bufferAir") = {3};

The mesh is created with <br> **gmsh -3 -o air_ground.msh air_ground.geo** 

<a id = 'LoadMesh'></a> 
# Loading mesh into *esys-escript*

In [54]:
from esys.escript import *
import esys.finley as finley
domain = finley.ReadGmsh("cubepygmsh.msh", numDim=3)

<a id = 'mkFly'></a>
## make a fly file
faster loading

In [55]:
#!/usr/bin/python3
from esys.finley import ReadGmsh
'''
converts meshname.msh to meshname.fly
run-escript mkfly.py meshname
'''

fname = "cubepygmsh"
mesh_file = fname+'.msh'
fly_file = fname+".fly"
 
print("converting "+mesh_file+" to "+fly_file)
dom = ReadGmsh(mesh_file, numDim=3) 
dom.write(fly_file)
print("finished")

converting cubepygmsh.msh to cubepygmsh.fly
finished


## loading fly files

In [56]:
from esys.escript import *
import esys.finley as finley
domain = finley.ReadMesh("cubepygmsh.fly", numDim=3)

<a id = 'diracMasks'></a>
## Making the fly, loading the fly with dirac points, using the tags for masks

In [57]:
#!/usr/bin/python3
from esys.finley import ReadGmsh
'''
converts meshname.msh to meshname.fly
run-escript mkfly.py meshname
'''

fname = "air_ground"
mesh_file = fname+'.msh'
fly_file = fname+".fly"
 
print("converting "+mesh_file+" to "+fly_file)
dom = ReadGmsh(mesh_file, numDim=3) 
dom.write(fly_file)
print("finished")

converting air_ground.msh to air_ground.fly
finished


### Masking and Dirac Points





from esys.escript import *
import esys.escript as escript
import esys.finley as finley
import numpy as np
from esys.escript import integrate, wherePositive, length
from esys.escript.pdetools import MaskFromTag

Sensors = [(10.,-100., 0.), (100., 100., 0.), (150., -500., 0.),
           (500.,-100., 0.), (-500., 120, 0.), (150.,-600., 0.),
           (300., -700., 0.), (100., 800., 0.), (150., 650., 0.),
           (-300., 1000., 0.)]

domain = ReadMesh("air_ground.fly", numDim = 3,
                      diracPoints = [sp for sp in Sensors], 
                      diracTags = [st for st in range(len(Sensors))])




# ReducedFunction
core = Scalar(0,ReducedFunction(domain))    
core.setTaggedValue("Base",1)
core.setTaggedValue("PaddingBase",1)
core.expand()

# Function
ground = Scalar(0,Function(domain))        
ground.setTaggedValue("coreGround",1)
ground.setTaggedValue("bufferGround",1)
ground.expand()

# continuous function
groundCF = MaskFromTag(domain,"DataArea")    
        self.mskBaseCF = MaskFromTag(self.domain,"Base")
        print("mskDataCF", self.mskDataCF.getNumberOfDataPoints())        
        print("mskBaseCF", self.mskBaseCF.getNumberOfDataPoints())

    # reduced function
        self.mskBaseRF = Scalar(0, ReducedFunction(self.domain))
        self.mskBaseRF.setTaggedValue("Base", 1)
        self.mskBaseRF.expand()
        print("mskBaseRF", self.mskBaseRF.getNumberOfDataPoints())
        self.mskBaseF=interpolate(self.mskBaseRF,Function(domain))   
        print("mskBaseF", self.mskBaseF.getNumberOfDataPoints())
        
        self.mskDataRF = Scalar(0, ReducedFunction(self.domain))
        self.mskDataRF.setTaggedValue("DataArea", 1)
        self.mskDataRF.expand()
        print("mskDataRF", self.mskDataRF.getNumberOfDataPoints())
        self.mskDataF=interpolate(self.mskDataRF,Function(domain))   
        print("mskDataF", self.mskDataF.getNumberOfDataPoints())
        
        
        # continuous function
        self.mskDataCF = MaskFromTag(self.domain,"DataArea")    
        self.mskBaseCF = MaskFromTag(self.domain,"Base")
        print("mskDataCF", self.mskDataCF.getNumberOfDataPoints())        
        print("mskBaseCF", self.mskBaseCF.getNumberOfDataPoints())

 saveSilo("masks", BRF = self.mskBaseRF, DRF = self.mskDataRF, DCF = self.mskDataCF, BCF =  self.mskBaseCF)

